In [2]:
# data preprocessing
import pandas as pd
import numpy as np
# data visualization
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import matplotlib as mpl
from wordcloud import WordCloud

# sklearn 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans

import re
import ast # 형변환 

from pprint import pprint as pp
import warnings
warnings.filterwarnings(action='ignore')

# 텍스트 마이닝
from eunjeon import Mecab
import nltk
from konlpy.tag import Okt
from nltk import FreqDist
from gensim.models import Word2Vec

In [3]:
# font
from matplotlib import font_manager, rc
import platform

if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
else:    
# Mac 인 경우
    rc('font', family='AppleGothic')
    
matplotlib.rcParams['axes.unicode_minus'] = False   

In [4]:
# import data
df_raw = pd.read_csv("./EDA_03")

# 활용 데이터 프레임화 
df = df_raw
# del df['Unnamed: 0']
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 324702 entries, 0 to 324701
Data columns (total 36 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   desertionno             324702 non-null  int64  
 1   popfile                 324702 non-null  object 
 2   happendt                324702 non-null  object 
 3   happenplace             324702 non-null  object 
 4   knm                     297107 non-null  object 
 5   colorcd                 324656 non-null  object 
 6   age                     324600 non-null  float64
 7   weight                  324569 non-null  object 
 8   noticeno                324701 non-null  object 
 9   noticesdt               324702 non-null  object 
 10  noticeedt               324702 non-null  object 
 11  sexcd                   324702 non-null  int64  
 12  neuteryn                324702 non-null  int64  
 13  specialmark             324702 non-null  object 
 14  carenm              

In [5]:
df.columns

Index(['desertionno', 'popfile', 'happendt', 'happenplace', 'knm', 'colorcd',
       'age', 'weight', 'noticeno', 'noticesdt', 'noticeedt', 'sexcd',
       'neuteryn', 'specialmark', 'carenm', 'orgnm', 'processstate', 'type',
       'lat', 'long', 'processstate_adopt', 'processstate_euthanize',
       'blc_col', 'brown_col', 'white_col', 'grey_col', 'yellow_col', 'kindcd',
       'weight_float', 'noticesdt_year', 'noticesdt_month', 'noticesdt_day',
       'good_behavior', 'sensitive', 'disease', 'accident'],
      dtype='object')

In [6]:

# df = df.drop(['desertionno', 'popfile', 'happendt', 'happenplace', 'knm', 'colorcd',
#         'noticeno', 'noticesdt', 'noticeedt', 
#        'specialmark', 'carenm', 'orgnm',  'type', 'weight'
#        ], axis = 1)

In [7]:
# df = df.drop(['processstate'], axis=1)

In [8]:
# df

In [9]:
df2 = df[['lat','long', 'desertionno']]

In [10]:
k = df2.head(10000)
k.reset_index
k

,lat,long,desertionno
0,35.228545,128.889352,448535202001458
1,34.486871,126.263485,446500202000046
2,34.486871,126.263485,446500202000045
3,34.486871,126.263485,446500202000044
4,34.311060,126.755054,446499202000215
...,...,...,...
9995,35.520461,127.725176,448546202000206
9996,34.837672,127.892423,448543202000280
9997,34.973149,128.322246,448542202000621
9998,34.973149,128.322246,448542202000620


In [11]:
k = k.groupby(['lat','long'])

In [12]:
k = k.count()


In [13]:
k.reset_index()


,lat,long,desertionno
0,33.489011,126.498302,686
1,34.311060,126.755054,28
2,34.486871,126.263485,8
3,34.573252,126.598927,38
4,34.611222,127.284978,32
...,...,...,...
224,38.105648,127.708049,10
225,38.110070,127.989882,6
226,38.146609,127.313226,53
227,38.207015,128.591849,19


In [14]:
k.rename(columns = {"desertionno": "count"}, inplace = True)
k

,,count
lat,long,
33.489011,126.498302,686
34.311060,126.755054,28
34.486871,126.263485,8
34.573252,126.598927,38
34.611222,127.284978,32
...,...,...
38.105648,127.708049,10
38.110070,127.989882,6
38.146609,127.313226,53


In [15]:
k = k.reset_index()

In [16]:
lat = k.lat.mean()
long = k.long.mean()

In [17]:
import folium

#지도의 중심을 지정하기 위해 위도와 경도의 평균 구하기
lat = k['lat'].mean()
long = k['long'].mean()

from folium.plugins import HeatMap
 

m = folium.Map(
    location = [lat, long],
    zoom_start = 8,
    tiles = 'Cartodb Positron'
)
 
for i in range(0, len(k)):
    folium.Circle(
        location = [k.lat[i],k.long[i]],
        color = '#000000',
        radius = 50, 
        fill = 'crimson',
    ).add_to(m)
    
m.save('map1.html')

In [19]:
m

In [ ]:
from folium.plugins import HeatMap
 
m = folium.Map(
    location = [36.5053542, 127.7043419],
    zoom_start = 8,
    tiles = 'Cartodb Positron'
)
 
HeatMap(k).add_to(m)
m.save('map.html')

In [23]:
from folium.plugins import HeatMap
 
u = folium.Map(
    location = [36.5053542, 127.7043419],
    zoom_start = 8,
    tiles = 'Cartodb Positron'
)
 
HeatMap(k).add_to(u)
u.save('map.html')

In [24]:
u

In [ ]:
m